# Scopes of Variables

This is the original code to calculate the area with Trapecio Method

In [1]:
%%writefile trap.c
#include <stdio.h>
#include <stdlib.h>
#include <omp.h>

double f(double x){return x * x;}
void trap(double a, double b, int n, double *global_result_p);

int main(){
    double global_result  = 0.0;
    double a = 0.0, b = 3.0;
    int n = 1073741824;
    int thread_count = 50;

    #pragma omp parallel num_threads(thread_count)
    {
        trap(a, b, n, &global_result);
    }
    printf("%.14e \n", global_result);
    return 0;

}
void trap(double a, double b, int n, double *global_result_p){
    double h, x, my_result;
    double local_a, local_b;
    int i, local_n;
    int my_rank = omp_get_thread_num();
    int thread_count = omp_get_num_threads();

    h = (b - a) / n;
    local_n = n / thread_count;
    local_a = a + my_rank * local_n * h;
    local_b = local_a + local_n * h;
    my_result = (f(local_a) + f(local_b)) / 2.0;
    for (i = 1; i <= local_n - 1; i++){
        x = local_a + i * h;
        my_result += f(x);
    } 
    my_result *= h;
    #pragma omp critical 
    *global_result_p += my_result;
} /* Trap */


Writing trap.c


In [2]:
!g++ -fopenmp trap.c
!./a.out

8.99999939650358e+00 


## First use of Scope


In [8]:
%%writefile trap1.c
#include <stdio.h>
#include <stdlib.h>
#include <omp.h>

double f(double x){return x * x;}
double trap(double a, double b, int n);

int main(){
    double global_result  = 0.0;
    double a = 0.0, b = 3.0;
    int n = 1073741824;
    int thread_count = 50;

    #pragma omp parallel num_threads(thread_count)
    {
      #pragma omp critical
      global_result += trap(a, b, n);
    }
    printf("%.14e \n", global_result);
    return 0;

}
double trap(double a, double b, int n){
    double h, x, my_result;
    double local_a, local_b;
    int i, local_n;
    int my_rank = omp_get_thread_num();
    int thread_count = omp_get_num_threads();

    h = (b - a) / n;
    local_n = n / thread_count;
    local_a = a + my_rank * local_n * h;
    local_b = local_a + local_n * h;
    my_result = (f(local_a) + f(local_b)) / 2.0;
    for (i = 1; i <= local_n - 1; i++){
        x = local_a + i * h;
        my_result += f(x);
    } 
    return my_result *= h;
} /* Trap */

Overwriting trap1.c


In [16]:
!g++ -fopenmp trap1.c -o trap1.exe
!./trap1.exe

8.99999939650358e+00 


## Second use of Scope

In [11]:
%%writefile trap2.c
#include <stdio.h>
#include <stdlib.h>
#include <omp.h>

double f(double x){return x * x;}
double trap(double a, double b, int n);

int main(){
    double global_result  = 0.0;
    double a = 0.0, b = 3.0;
    int n = 1073741824;
    int thread_count = 50;

    #pragma omp parallel num_threads(thread_count)
    {
      double my_result = 0.0;
      my_result += trap(a, b, n);
      #pragma omp critical
      global_result += my_result;
    }
    printf("%.14e \n", global_result);
    return 0;

}
double trap(double a, double b, int n){
    double h, x, my_result;
    double local_a, local_b;
    int i, local_n;
    int my_rank = omp_get_thread_num();
    int thread_count = omp_get_num_threads();

    h = (b - a) / n;
    local_n = n / thread_count;
    local_a = a + my_rank * local_n * h;
    local_b = local_a + local_n * h;
    my_result = (f(local_a) + f(local_b)) / 2.0;
    for (i = 1; i <= local_n - 1; i++){
        x = local_a + i * h;
        my_result += f(x);
    } 
    return my_result *= h;
} /* Trap */

Writing trap2.c


In [17]:
!g++ -fopenmp trap2.c -o trap2.exe
!./trap2.exe

8.99999939650358e+00 


## Third use of Scope

In [13]:
%%writefile trap3.c
#include <stdio.h>
#include <stdlib.h>
#include <omp.h>

double f(double x){return x * x;}
double trap(double a, double b, int n);

int main(){
    double global_result  = 0.0;
    double a = 0.0, b = 3.0;
    int n = 1073741824;
    int thread_count = 50;

    #pragma omp parallel num_threads(thread_count) reduction(+: global_result)
    {
      global_result += trap(a, b, n);
    }
    printf("%.14e \n", global_result);
    return 0;

}
double trap(double a, double b, int n){
    double h, x, my_result;
    double local_a, local_b;
    int i, local_n;
    int my_rank = omp_get_thread_num();
    int thread_count = omp_get_num_threads();

    h = (b - a) / n;
    local_n = n / thread_count;
    local_a = a + my_rank * local_n * h;
    local_b = local_a + local_n * h;
    my_result = (f(local_a) + f(local_b)) / 2.0;
    for (i = 1; i <= local_n - 1; i++){
        x = local_a + i * h;
        my_result += f(x);
    } 
    return my_result *= h;
} /* Trap */

Writing trap3.c


In [18]:
!g++ -fopenmp trap3.c -o trap3.exe
!./trap3.exe

8.99999939650358e+00 
